In [ ]:
# install avalanche
!pip install avalanche-lib==0.2.1

# Example 

**Avalanche datasets**: 
```
from avalanche.benchmarks.datasets import MNIST, FashionMNIST, KMNIST, EMNIST, \
    QMNIST, FakeData, CocoCaptions, CocoDetection, LSUN, ImageNet, CIFAR10, \
    CIFAR100, STL10, SVHN, PhotoTour, SBU, Flickr8k, Flickr30k, VOCDetection, \
    VOCSegmentation, Cityscapes, SBDataset, USPS, Kinetics400, HMDB51, UCF101, \
    CelebA, CORe50Dataset, TinyImagenet, CUB200, OpenLORIS, MiniImageNetDataset, Stream51, \
    CLEARDataset
```



----------

**Avalanche Benchmark**: can collect stream data (newest data); and profrom training and testing at the same time

In [ ]:
# import avalanche benchmarks
from avalanche.benchmarks.classic import CORe50, SplitTinyImageNet, SplitCIFAR10, \
    SplitCIFAR100, SplitCIFAR110, SplitMNIST, RotatedMNIST, PermutedMNIST, SplitCUB200

In [ ]:
# creating the benchmark (scenario object)
perm_mnist = PermutedMNIST(
    n_experiences=3,
    seed=1234,
)

In [ ]:
# recovering the train and test streams
train_stream = perm_mnist.train_stream
test_stream = perm_mnist.test_stream

In [ ]:
# iterating over the train stream
for experience in train_stream:
    print("Start of task ", experience.task_label)
    print('Classes in this task:', experience.classes_in_this_experience)

    # The current Pytorch training set can be easily recovered through the 
    # experience
    current_training_set = experience.dataset
    # ...as well as the task_label
    print('Task {}'.format(experience.task_label))
    print('This task contains', len(current_training_set), 'training examples')

    # we can recover the corresponding test experience in the test stream
    current_test_set = test_stream[experience.current_experience].dataset
    print('This task contains', len(current_test_set), 'test examples')

Start of task  0
Classes in this task: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Task 0
This task contains 60000 training examples
This task contains 10000 test examples
Start of task  1
Classes in this task: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Task 1
This task contains 60000 training examples
This task contains 10000 test examples
Start of task  2
Classes in this task: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Task 2
This task contains 60000 training examples
This task contains 10000 test examples


---------


**Benchmark generators**

In [ ]:
from avalanche.benchmarks.generators import nc_benchmark, ni_benchmark
from torchvision.datasets import MNIST

In [ ]:
mnist_train = MNIST('.', train=True, download=True)
mnist_test = MNIST('.', train=False)

# create benchmarks for MNIST dataset
benchmark = ni_benchmark(
    mnist_train, mnist_test, n_experiences=10, shuffle=True, seed=1234,
    balance_experiences=True
)
benchmark = nc_benchmark(
    mnist_train, mnist_test, n_experiences=10, shuffle=True, seed=1234,
    task_labels=False
)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



In [ ]:
from avalanche.models import SimpleMLP
from avalanche.training import Naive, CWRStar, Replay, GDumb, \
    Cumulative, LwF, GEM, AGEM, EWC, AR1
from torch.optim import SGD
from torch.nn import CrossEntropyLoss

In [ ]:
# strategy
model = SimpleMLP(num_classes=10)
cl_strategy = Naive(
    model, SGD(model.parameters(), lr=0.001, momentum=0.9),
    CrossEntropyLoss(), train_mb_size=100, train_epochs=4, eval_mb_size=100
)

-----------

**Create your own strategy**

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
class MyStrategy():
    """My Basic Strategy"""

    def __init__(self, model, optimizer, criterion):
        self.model = model
        self.optimizer = optimizer
        self.criterion = criterion

    def train(self, experience):
        # here you can implement your own training loop for each experience (i.e. 
        # batch or task).

        train_dataset = experience.dataset
        t = experience.task_label
        train_data_loader = DataLoader(
            train_dataset, num_workers=4, batch_size=128
        )

        for epoch in range(1):
            for mb in train_data_loader:
                # you magin here...
                pass

    def eval(self, experience):
        # here you can implement your own eval loop for each experience (i.e. 
        # batch or task).

        eval_dataset = experience.dataset
        t = experience.task_label
        eval_data_loader = DataLoader(
            eval_dataset, num_workers=4, batch_size=128
        )

In [ ]:
from avalanche.models import SimpleMLP
from avalanche.benchmarks import SplitMNIST

In [ ]:
# Benchmark creation
benchmark = SplitMNIST(n_experiences=5)

# Model Creation
model = SimpleMLP(num_classes=benchmark.n_classes)

# Create the Strategy Instance (MyStrategy)
cl_strategy = MyStrategy(
    model, SGD(model.parameters(), lr=0.001, momentum=0.9),
    CrossEntropyLoss())

# Training Loop
print('Starting experiment...')

for exp_id, experience in enumerate(benchmark.train_stream):
    print("Start of experience ", experience.current_experience)

    cl_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the current test set')
    cl_strategy.eval(benchmark.test_stream[exp_id])

Starting experiment...
Start of experience  0


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Training completed
Computing accuracy on the current test set
Start of experience  1
Training completed
Computing accuracy on the current test set
Start of experience  2
Training completed
Computing accuracy on the current test set
Start of experience  3
Training completed
Computing accuracy on the current test set
Start of experience  4
Training completed
Computing accuracy on the current test set


--------

**Evaluation**

In [ ]:
# utility functions to create plugin metrics
from avalanche.evaluation.metrics import accuracy_metrics, loss_metrics, forgetting_metrics
from avalanche.logging import InteractiveLogger, TensorboardLogger
from avalanche.training.plugins import EvaluationPlugin

In [ ]:
eval_plugin = EvaluationPlugin(
    # accuracy after each training epoch
    # and after each evaluation experience
    accuracy_metrics(epoch=True, experience=True),
    # loss after each training minibatch and each
    # evaluation stream
    loss_metrics(minibatch=True, stream=True),
    # catastrophic forgetting after each evaluation
    # experience
    forgetting_metrics(experience=True, stream=True), 
    # add as many metrics as you like
    loggers=[InteractiveLogger(), TensorboardLogger()])

# pass the evaluation plugin instance to the strategy
# strategy = EWC(..., evaluator=eval_plugin)

/usr/local/lib/python3.7/dist-packages/avalanche/training/plugins/evaluation.py:82: UserWarning: No benchmark provided to the evaluation plugin. Metrics may be computed on inconsistent portion of streams, use at your own risk.
  "No benchmark provided to the evaluation plugin. "


-----------

**Example**</br>
Ref: https://avalanche.continualai.org/getting-started/learn-avalanche-in-5-minutes#benchmarks-generators

In [ ]:
from avalanche.benchmarks.classic import SplitMNIST
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics,\
    loss_metrics, timing_metrics, cpu_usage_metrics, StreamConfusionMatrix,\
    disk_usage_metrics, gpu_usage_metrics
from avalanche.models import SimpleMLP
from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger
from avalanche.training.plugins import EvaluationPlugin
from avalanche.training import Naive

from torch.optim import SGD
from torch.nn import CrossEntropyLoss

benchmark = SplitMNIST(n_experiences=5)

# MODEL CREATION
model = SimpleMLP(num_classes=benchmark.n_classes)

# DEFINE THE EVALUATION PLUGIN and LOGGERS
# The evaluation plugin manages the metrics computation.
# It takes as argument a list of metrics, collectes their results and returns 
# them to the strategy it is attached to.

# log to Tensorboard
tb_logger = TensorboardLogger()

# log to text file
text_logger = TextLogger(open('log.txt', 'a'))

# print to stdout
interactive_logger = InteractiveLogger()

eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True),
    cpu_usage_metrics(experience=True),
    forgetting_metrics(experience=True, stream=True),
    StreamConfusionMatrix(num_classes=benchmark.n_classes, save_image=False),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loggers=[interactive_logger, text_logger, tb_logger],
    benchmark=benchmark
)

# CREATE THE STRATEGY INSTANCE (NAIVE)
cl_strategy = Naive(
    model, SGD(model.parameters(), lr=0.001, momentum=0.9),
    CrossEntropyLoss(), train_mb_size=500, train_epochs=1, eval_mb_size=100,
    evaluator=eval_plugin)

# TRAINING LOOP
print('Starting experiment...')
results = []
for experience in benchmark.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy.train(experience, num_workers=4)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # eval also returns a dictionary which contains all the metric values
    results.append(cl_strategy.eval(benchmark.test_stream, num_workers=4))

Starting experiment...
Start of experience:  0
Current Classes:  [8, 7]
-- >> Start of training phase << --


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


100%|██████████| 25/25 [00:04<00:00,  5.20it/s]
Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 120530.6543
	DiskUsage_MB/train_phase/train_stream/Task000 = 120530.6543
	Loss_Epoch/train_phase/train_stream/Task000 = 1.2330
	Loss_MB/train_phase/train_stream/Task000 = 0.3037
	Time_Epoch/train_phase/train_stream/Task000 = 4.6564
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.6793
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.9569
-- >> End of training phase << --
Training completed
Computing accuracy on the whole test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 21/21 [00:01<00:00, 11.32it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Task000/Exp000 = 27.5165
	DiskUsage_Exp/eval_phase/test_stream/Task000/Exp000 = 120531.1816
	Loss_Exp/eval_phase/test_stream/Task000/Exp000 = 0.2961
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.9500
-

In [ ]:
!tensorboard --logdir logs/fit

2022-11-15 23:57:48.362041: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.9.1 at http://localhost:6006/ (Press CTRL+C to quit)
Error in atexit._run_exitfuncs:
Traceback (most recent call last):
  File "/usr/lib/python3.7/logging/__init__.py", line 2021, in shutdown
    def shutdown(handlerList=_handlerList):
KeyboardInterrupt


-------------

# Continual learning using avalanche library

**Continual learning**: it's approch that it keep updating the model with newest data.

**Avalanche**: a tool can perform continual learning. It has main components: 
```
Avalanche
├── Benchmarks (handle data preprocessing)
│   ├── Classic
│   ├── Datasets (use well-known datasets)
│   └── Generators (collect stream data)
├── Evaluation 
│   ├── Metrics (accuracy checking)
│   └── Tensorboard (check model's performance)
├── Training 
│   └── Strategies (pipeline: model+optimizer+loss+evaluator)
├── Models
└── Loggers (for visualization)
```



In [24]:
!pip install avalanche-lib==0.2.1

In [ ]:
# use an existing model, SimpleCNN
from avalanche.models import SimpleCNN
from avalanche.models import SimpleMLP
from avalanche.models import SimpleMLP_TinyImageNet
from avalanche.models import MobilenetV1

model = SimpleCNN()
print(model)

---------

In [26]:
# use an existing model, IncrementalClassifier
# this time we add benchmark (for dynamic model expansion)
from avalanche.benchmarks import SplitMNIST
from avalanche.models import IncrementalClassifier

benchmark = SplitMNIST(5, shuffle=False)
model = IncrementalClassifier(in_features=784)

print(model)
for exp in benchmark.train_stream:
    model.adaptation(exp)
    print(model)

IncrementalClassifier(
  (classifier): Linear(in_features=784, out_features=2, bias=True)
)
IncrementalClassifier(
  (classifier): Linear(in_features=784, out_features=2, bias=True)
)
IncrementalClassifier(
  (classifier): Linear(in_features=784, out_features=4, bias=True)
)
IncrementalClassifier(
  (classifier): Linear(in_features=784, out_features=6, bias=True)
)
IncrementalClassifier(
  (classifier): Linear(in_features=784, out_features=8, bias=True)
)
IncrementalClassifier(
  (classifier): Linear(in_features=784, out_features=10, bias=True)
)


------

In [21]:
# use an existing multitask model/classifier
from avalanche.benchmarks import SplitMNIST
from avalanche.models import MultiHeadClassifier

benchmark = SplitMNIST(5, shuffle=False, return_task_id=True)
model = MultiHeadClassifier(in_features=784)

print(model)
for exp in benchmark.train_stream:
    model.adaptation(exp)
    print(model)

MultiHeadClassifier(
  (classifiers): ModuleDict(
    (0): IncrementalClassifier(
      (classifier): Linear(in_features=784, out_features=2, bias=True)
    )
  )
)
MultiHeadClassifier(
  (classifiers): ModuleDict(
    (0): IncrementalClassifier(
      (classifier): Linear(in_features=784, out_features=2, bias=True)
    )
  )
)
MultiHeadClassifier(
  (classifiers): ModuleDict(
    (0): IncrementalClassifier(
      (classifier): Linear(in_features=784, out_features=2, bias=True)
    )
    (1): IncrementalClassifier(
      (classifier): Linear(in_features=784, out_features=2, bias=True)
    )
  )
)
MultiHeadClassifier(
  (classifiers): ModuleDict(
    (0): IncrementalClassifier(
      (classifier): Linear(in_features=784, out_features=2, bias=True)
    )
    (1): IncrementalClassifier(
      (classifier): Linear(in_features=784, out_features=2, bias=True)
    )
    (2): IncrementalClassifier(
      (classifier): Linear(in_features=784, out_features=2, bias=True)
    )
  )
)
MultiHeadClas

--------

In [22]:
# or we can define our own multitask model
from avalanche.models import MultiTaskModule

class CustomMTModule(MultiTaskModule):
    def __init__(self, in_features, initial_out_features=2):
        super().__init__()

    def adaptation(self, dataset):
        super().adaptation(dataset)
        # your adaptation goes here

    def forward_single_task(self, x, task_label):
        # your forward goes here.
        # task_label is a single integer
        # the mini-batch is split by task-id inside the forward method.
        pass

---------

In [27]:
# convert regular model to multitask learning model
from avalanche.models import as_multitask

model = SimpleCNN()
print(model)

mt_model = as_multitask(model, 'classifier')
print(mt_model)

SimpleCNN(
  (features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Dropout(p=0.25, inplace=False)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Dropout(p=0.25, inplace=False)
    (12): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
    (13): ReLU(inplace=True)
    (14): AdaptiveMaxPool2d(output_size=1)
    (15): Dropout(p=0.25, inplace=False)
  )
  (classifier): Sequential(
    (0): Linear(in_features=64, out_features=10, bias=True)
  )
)
MultiTaskDecorator(
  (model): SimpleCNN(
    (features): Seque

-------